In [3]:
import pandas as pd
from typing import Optional
from restr_rating.exception import  RatingException
from restr_rating.logger import logging

In [2]:
df = pd.read_csv('zomato.csv')

In [4]:
from restr_rating.entity import config_entity, artifact_entity

In [5]:
from restr_rating.entity.config_entity import TrainingPipelineConfig, DataIngestionConfig
from restr_rating.entity.artifact_entity import DataIngestionArtifact, DataValidationArtifact

In [6]:
training_pipeline_config = TrainingPipelineConfig()
data_ingestion_config = DataIngestionConfig(training_pipeline_config)

In [7]:
train = data_ingestion_config.train_file_name

In [8]:
train

'/config/workspace/artifact/02_14_2023__03_25_49/data_ingestion/dataset/train.csv'

In [9]:
df = pd.read_csv("/config/workspace/artifact/02_14_2023__03_10_38/data_ingestion/dataset/train.csv")

In [11]:
null_report = df.isna().sum()/df.shape[0]
null_report

url                            0.000000
address                        0.000000
name                           0.000000
online_order                   0.000000
book_table                     0.000000
rate                           0.149735
votes                          0.000000
phone                          0.023228
location                       0.000435
rest_type                      0.004254
dish_liked                     0.543301
cuisines                       0.000943
approx_cost(for two people)    0.006550
reviews_list                   0.000000
menu_item                      0.000000
listed_in(type)                0.000000
listed_in(city)                0.000000
dtype: float64